# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import folium

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,weno,7.4515,151.8468,28.97,84,75,2.06,FM,1692710212
1,1,andahuaylas,-13.6556,-73.3872,12.38,68,22,1.72,PE,1692710212
2,2,adamstown,-25.0660,-130.1015,18.86,68,100,13.55,PN,1692710212
3,3,west island,-12.1568,96.8225,26.99,78,100,10.29,CC,1692710129
4,4,bredasdorp,-34.5322,20.0403,17.79,57,2,13.59,ZA,1692710213


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
import folium
from branca.colormap import LinearColormap

# Create a map centered at an arbitrary location (e.g., the first city's coordinates)
m = folium.Map(location=[city_data_df['Lat'].iloc[0], city_data_df['Lng'].iloc[0]], zoom_start=4)

# Define a colormap for humidity values
colormap = LinearColormap(['blue', 'green', 'yellow', 'orange', 'red'], vmin=city_data_df['Humidity'].min(), vmax=city_data_df['Humidity'].max())

# Add markers for each city with different colors based on humidity
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 9,  # Adjust the scaling for appropriate marker size
        color=colormap(row['Humidity']),
        fill=True,
        fill_color=colormap(row['Humidity']),
        fill_opacity=0.6,
        popup=row['City']
    ).add_to(m)

# Add the legend to the map
colormap.caption = 'Humidity Levels'
m.add_child(colormap)

# Display the map
m



### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[
    (city_data_df['Wind Speed'] <= 6.7) &
    (city_data_df['Max Temp'] >= 25) &  
    (city_data_df['Cloudiness'] == 0)
].copy()  # Create a copy of the filtered DataFrame

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data
filtered_cities_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,tsiombe,-25.3000,45.4833,29.19,39,0,5.67,MG,1692710214
40,40,crucecita,-28.9500,-58.9833,26.29,51,0,5.94,AR,1692710219
60,60,saquarema,-22.9200,-42.5103,27.08,62,0,1.45,BR,1692710176
68,68,sao joao da barra,-21.6403,-41.0511,27.05,66,0,0.36,BR,1692710226
70,70,saipan,15.1355,145.7010,28.39,100,0,0.00,MP,1692710227


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,tsiombe,MG,-25.3000,45.4833,39,
40,crucecita,AR,-28.9500,-58.9833,51,
60,saquarema,BR,-22.9200,-42.5103,62,
68,sao joao da barra,BR,-21.6403,-41.0511,66,
70,saipan,MP,15.1355,145.7010,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000  # 100,000 meters
params = {
    "categories": "accommodation.hotel",
    "radius": radius,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting Hotel Search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"
    
    # Make an API request using the params dictionary
    response = requests.get(f"{base_url}/nearby", params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["features"][0]["properties"]["formatted"]
        hotel_df.loc[index, "Hotel Lat"] = name_address["features"][0]["geometry"]["coordinates"][1]
        hotel_df.loc[index, "Hotel Lng"] = name_address["features"][0]["geometry"]["coordinates"][0]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No Hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting Hotel Search
tsiombe - nearest hotel: No Hotel found
crucecita - nearest hotel: No Hotel found
saquarema - nearest hotel: Pousada Garota de Itaúna
sao joao da barra - nearest hotel: Pousada Mediterrâneo
saipan - nearest hotel: Chalan Kanoa Beach Hotel
sangin - nearest hotel: No Hotel found
tarasovskiy - nearest hotel: No Hotel found
ulundi - nearest hotel: Protea Hotel
razdol'noye - nearest hotel: Портовое
guajara mirim - nearest hotel: Novo Hotel Campos
anqing - nearest hotel: 安庆大酒店
kas - nearest hotel: KEKOVA OTEL
thabazimbi - nearest hotel: Maroela Guest Lodge
tete - nearest hotel: Zambezi Hotel
montepuez - nearest hotel: Complexo Messalo
cordele - nearest hotel: Baymont by Wyndham Cordele
turpan - nearest hotel: Oriental
frankfort - nearest hotel: No Hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
greenville - nearest hotel: Quality Inn
midland - nearest hotel: DoubleTree by Hilton Hotel Midland Plaza
kasane - nearest hotel: Kasane Travel Lodge
hasaki - neares

,City,Country,Lat,Lng,Humidity,Hotel Name,Hotel Address,Hotel Lat,Hotel Lng
10,tsiombe,MG,-25.3000,45.4833,39,No Hotel found,NaN,NaN,NaN
40,crucecita,AR,-28.9500,-58.9833,51,No Hotel found,NaN,NaN,NaN
60,saquarema,BR,-22.9200,-42.5103,62,Pousada Garota de Itaúna,"Pousada Garota de Itaúna, Avenida Oceânica, It...",-22.935030,-42.483201
68,sao joao da barra,BR,-21.6403,-41.0511,66,Pousada Mediterrâneo,"Pousada Mediterrâneo, Rua Feliciano Sodré, Car...",-21.642192,-41.019011
70,saipan,MP,15.1355,145.7010,100,Chalan Kanoa Beach Hotel,"Chalan Kanoa Beach Hotel, Atuhong Place, Chala...",15.143300,145.699681


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:
import folium
from branca.colormap import LinearColormap

# Create a map centered at an arbitrary location (e.g., the first city's coordinates)
m = folium.Map(location=[hotel_df['Lat'].iloc[0], hotel_df['Lng'].iloc[0]], zoom_start=4)

# Define a colormap for humidity values
colormap = LinearColormap(['blue', 'green', 'yellow', 'orange', 'red'], vmin=hotel_df['Humidity'].min(), vmax=hotel_df['Humidity'].max())

# Iterate through the DataFrame and add markers only for rows with a hotel name
for index, row in hotel_df.iterrows():
    if not pd.isna(row['Hotel Name']):  # Check if 'Hotel Name' is not NaN
        folium.CircleMarker(
            location=[row['Lat'], row['Lng']],
            radius=row['Humidity'] / 5,  # Adjust the scaling for appropriate marker size
            color=colormap(row['Humidity']),
            fill=True,
            fill_color=colormap(row['Humidity']),
            fill_opacity=0.6,
            popup=f"City: {row['City']}<br>Country: {row['Country']}<br>Hotel: {row['Hotel Name']}"
        ).add_to(m)

# Add the legend to the map
colormap.caption = 'Humidity Levels'
m.add_child(colormap)

# Display the map
m



# Save the map to an HTML file
m.save("../output_data/map_with_hotels.html")

